In [13]:
import httpx
import html
import re


In [59]:
from langchain_core.tools import tool


def get_wikipedia_summary(query: str) -> str:
    """ this function is used to get the wikipedia summary of the query"""
    response = httpx.get(
        "https://en.wikipedia.org/w/api.php",
        params={
            "action": "query",
            "format": "json",
            "list": "search",
            "srsearch": query
        },
        headers={
            "User-Agent": "KhushilApp/1.0 (https://example.com/contact)"
        }
    )

    response.raise_for_status()
    text = ""
    for index,search in enumerate(response.json()['query']['search']):
        raw_snippet = search['snippet']
        cleaned_snippet = re.sub(r'<[^>]+>', '', raw_snippet)
        text += f"Search result {index+1}: {html.unescape(cleaned_snippet)}\n"
    return text
    

# print(get_wikipedia_summary("AI agents"))

In [60]:

def math_calculator(query:str)->str:
    """ this function is used to calculate the result of the query"""
    try:
        result = eval(query)
        return f"The result of {query} is {result}"
    except Exception as e:
        print("error -->",e)
        return f"Invalid input: `{query}` can not be calculated"

# print(math_calculator("1+2/5%6-5"))
# print(math_calculator("hi"))

In [61]:

def fetch_todos() -> str:
    """ this function is used to fetch the todos from the json placeholder"""
    try:
        response = httpx.get("https://jsonplaceholder.typicode.com/todos")
        if response.status_code != 200:
            return "Failed to fetch todos"
        todos = response.json()
        return f"Here are the todos: {todos}"
    except Exception as e:
        print("error -->", e)
        return "Failed to fetch todos"

# print(fetch_todos())

In [ ]:
from langchain_core.messages import HumanMessage, SystemMessage
from langchain_ollama import ChatOllama
class Chat:
    def __init__(self,system_prompt:str , model:str="gpt-oss:20b-cloud"):
        self.system_prompt = system_prompt
        self.model = model
        self.messages = []
        self.chat = ChatOllama(model=self.model)
        
        if system_prompt:
            self.messages.append(SystemMessage(content=system_prompt))
            self.chat.invoke([SystemMessage(content=system_prompt)])
            
    def __call__(self,message:str)->str:
        self.messages.append(HumanMessage(content=message))
        response = self.chat.invoke(self.messages)
        self.messages.append(response)
        return response.content
    
    def show_messages(self)->str:
        return "\n".join([m.content for m in self.messages])
    
    def __str__(self)->str:
        return f" Chat with {self.model} model"
    
Ai_Chat = Chat(system_prompt="You are a helpful assistant",model="qwen3-vl:235b-instruct-cloud")
print(Ai_Chat("What is the capital of India?"))
print(Ai_Chat("Explain me AI agents in short"))
Ai_Chat.show_messages()
print(Ai_Chat)

In [148]:
from langchain_core.messages import AIMessage, HumanMessage, SystemMessage, ToolMessage
from langchain_ollama import ChatOllama


def get_message_role(message):
    if type(message) == SystemMessage:
        return "System"
    elif type(message) == AIMessage:
        return "AI"
    elif type(message) == HumanMessage:
        return "User"
    elif type(message) == ToolMessage:
        return "Tool"


class Agent:
    def __init__(self, model: str = "gpt-oss:20b-cloud"):
        self.model = model
        self.system_prompt = """
        You are a helpful assistant that can use the following tools:
        - get_wikipedia_summary , AFTER TOOL CALL WHEN YOU GET THE RESULT FROM THE TOOL, rephrase the final result to the user in a concise manner
        - math_calculator , AFTER TOOL CALL WHEN YOU GET THE RESULT FROM THE TOOL ,after you get the result from the tool, you should return the result with calculation steps if possible to the user
        - fetch_todos , AFTER TOOL CALL WHEN YOU GET THE RESULT FROM THE TOOL ,format properly the todos to the user
        """
        self.messages = [SystemMessage(content=self.system_prompt)]
        # Create a tool mapping for easier access
        self.tools = {
            "get_wikipedia_summary": get_wikipedia_summary,
            "math_calculator": math_calculator,
            "fetch_todos": fetch_todos,
        }
        self.chat = ChatOllama(model=self.model).bind_tools(
            [get_wikipedia_summary, math_calculator, fetch_todos]
        )

    def __call__(self, message: str) -> str:
        
        self.messages.append(HumanMessage(content=message))
        while True:
            response = self.chat.invoke(self.messages)
            self.messages.append(response)
            # Execute all tool calls
            if response.tool_calls:
                tool_name = response.tool_calls[0]["name"]
                tool_args = response.tool_calls[0]["args"]
                tool_call_id = response.tool_calls[0]["id"]

                print(f"Calling tool: {tool_name} with args: {tool_args}")

                # Get the tool function and call it
                tool_func = self.tools[tool_name]

                # Handle tools with different signatures
                if tool_name == "fetch_todos":
                    result = tool_func()
                else:
                    result = tool_func(**tool_args)

                # Append ToolMessage with tool_call_id (IMPORTANT!)
                self.messages.append(
                    ToolMessage(content=str(result), tool_call_id=tool_call_id)
                )
            else:
                break 
            
        
        # Get the next response from the model
        response = self.chat.invoke(self.messages)

        # Append final response and return
        self.messages.append(response)
            
        for m in self.messages:
            if type(m) != ToolMessage and m.content != "":
                print(f"{get_message_role(m)}: {m.content}", end="\n")
        return response.content

    def show_messages(self) -> str:
        return [{m.content} for m in self.messages]

    def __str__(self) -> str:
        return f"Agent with {self.model} model"

In [150]:
ai_agent = Agent()

In [151]:
ai_agent("whats 2 + 6 / 7 +90 % 50 and then also give me summary on AI agents and then fetch all todos for me")


Calling tool: math_calculator with args: {'query': '2 + 6 / 7 + 90 % 50'}
Calling tool: get_wikipedia_summary with args: {'query': 'AI agents'}
Calling tool: fetch_todos with args: {}
System: 
        You are a helpful assistant that can use the following tools:
        - get_wikipedia_summary , AFTER TOOL CALL WHEN YOU GET THE RESULT FROM THE TOOL, rephrase the final result to the user in a concise manner
        - math_calculator , AFTER TOOL CALL WHEN YOU GET THE RESULT FROM THE TOOL ,after you get the result from the tool, you should return the result with calculation steps if possible to the user
        - fetch_todos , AFTER TOOL CALL WHEN YOU GET THE RESULT FROM THE TOOL ,format properly the todos to the user
        
User: whats 2 + 6 / 7 +90 % 50 and then also give me summary on AI agents and then fetch all todos for me
AI: **1. Math result**

\(2 + \dfrac{6}{7} + (90 \bmod 50)\)

1. \(6 \div 7 = 0.857142857\)
2. \(90 \bmod 50 = 40\)
3. \(2 + 0.857142857 + 40 = 42.857142857\)


''

In [70]:
print(ai_agent)
ai_agent.show_messages()

Agent with gpt-oss:20b-cloud model


[{'\n        You are a helpful assistant that can use the following tools:\n        - get_wikipedia_summary\n        - math_calculator\n        - fetch_todos\n        '}]